데이터 출처 : https://www.kaggle.com/iabhishekofficial/mobile-price-classification?select=train.csv

In [2]:
!pip install xgboost

  Using cached xgboost-1.3.3-py3-none-win_amd64.whl (95.2 MB)


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [4]:
df = pd.read_csv('mobile_train.csv')
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [5]:
## 결과 변수 확인
df.price_range.value_counts()

0    500
1    500
2    500
3    500
Name: price_range, dtype: int64

In [6]:
## 데이터 분할
    # stratify = 결과변수 를 설정하여 비율에 맞게 분할되도록 설정
X_train, X_test, y_train, y_test = train_test_split(df.drop('price_range',axis=1),
                                                    df.price_range, 
                                                    random_state=0, 
                                                    test_size=.2,
                                                   stratify = df['price_range'])


In [7]:
y_test.value_counts()

0    100
1    100
2    100
3    100
Name: price_range, dtype: int64

In [ ]:
xgb.XGBClassifier(learning_rate=0.1,
                          n_estimators=220,
                          max_depth=10,
                          booster='dart',
                            random_state=42,
                          tree_method='exact',
                            objective='multiclass',
                            num_class=3,
                            n_jobs=-1)

In [9]:
## 모델링
import xgboost as xgb
from sklearn import metrics

clf = xgb.XGBClassifier(n_estimators=1000,
                        n_jobs = 10,
                        random_state = 0)
clf = clf.fit(X_train, y_train)

# predict
xgb_pred = clf.predict(X_test)

C:\Users\Roh Seung Chan\anaconda3\envs\book\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:26:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [11]:
# 정확도 확인
print('accuracy', metrics.accuracy_score(np.array(y_test), xgb_pred) )

accuracy 0.92


## GridSearchCV 사용
- GridSearch CV를 사용하여 랜덤 포레스트의 하이퍼 파라미터를 튜닝해보자

In [12]:
from sklearn.model_selection import GridSearchCV

param = {
    'n_estimators' : [100, 500, 1000, 2000],
    'max_depth' : [11,12,13,14,15],
    'early_stoppings' : [100,200] 
}

xgb_model = xgb.XGBClassifier(
                        random_state = 0)

grid_cv = GridSearchCV(xgb_model, param_grid=param, cv=5, verbose=1, n_jobs=10)
grid_cv.fit(X_train, y_train.values)
xgb_pred = grid_cv.predict(X_test)
print('최적 하이퍼 파라미터: \n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

# 정확도 확인
print('accuracy', metrics.accuracy_score(np.array(y_test), xgb_pred) )

Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\Users\Roh Seung Chan\anaconda3\envs\book\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:30:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { early_stoppings } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:30:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
최적 하이퍼 파라미터: 
 {'early_stoppings': 100, 'max_depth': 12, 'n_estimators': 500}
최고 예측 정확도: 0.9056
accuracy 0.9075
